In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from warnings import filterwarnings
filterwarnings('ignore')

In [5]:
def davies_bouldin_index(X, labels):
    """
    Calculate Davies-Bouldin Index
    Lower values indicate better clustering
    """
    n_clusters = len(np.unique(labels))
    cluster_centers = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])
    
    # Calculate cluster dispersions (average distance to center)
    dispersions = np.zeros(n_clusters)
    for i in range(n_clusters):
        if sum(labels == i) > 0:
            dispersions[i] = np.mean(cdist(X[labels == i], [cluster_centers[i]]))
    
    # Calculate Davies-Bouldin Index
    db_index = 0
    for i in range(n_clusters):
        if dispersions[i] == 0:
            continue
        max_ratio = 0
        for j in range(n_clusters):
            if i != j and dispersions[j] > 0:
                ratio = (dispersions[i] + dispersions[j]) / \
                        max(1e-5, np.linalg.norm(cluster_centers[i] - cluster_centers[j]))
                max_ratio = max(max_ratio, ratio)
        db_index += max_ratio
    
    return db_index / n_clusters

def prepare_features(customers_df, transactions_df):
    """
    Prepare features for clustering by combining customer profiles and transaction data
    """
    # First, let's identify numerical and categorical columns in customers_df
    # We'll drop CustomerID as it's not a feature for clustering
    customers_features = customers_df.drop('CustomerID', axis=1)
    
    # Handle categorical columns in customers_df
    categorical_columns = customers_features.select_dtypes(include=['object']).columns
    
    # Create dummy variables for categorical columns
    for col in categorical_columns:
        # For columns with many unique values, we might want to handle them differently
        if customers_features[col].nunique() < 10:  # For columns with few unique values
            customers_features = pd.get_dummies(customers_features, columns=[col], prefix=col)
        else:  # For columns with many unique values, we might want to drop them
            customers_features = customers_features.drop(col, axis=1)
    
    # Aggregate transaction data
    transaction_features = transactions_df.groupby('CustomerID').agg({
        'TransactionID': 'count',
        'TotalValue': ['sum', 'mean', 'std']
    }).reset_index()
    
    # Flatten column names
    transaction_features.columns = ['CustomerID', 'TransactionCount', 
                                  'TotalSpend', 'AverageSpend', 'SpendStd']
    
    # Merge customer features with transaction features
    features_df = customers_df[['CustomerID']].merge(
        transaction_features, 
        on='CustomerID', 
        how='left'
    )
    
    # Add the processed customer features
    for col in customers_features.columns:
        features_df[col] = customers_features[col]
    
    # Fill missing values
    features_df = features_df.fillna(0)
    
    # Store feature names for later use
    feature_names = features_df.columns[1:].tolist()  # Exclude CustomerID
    
    return features_df, feature_names

def find_optimal_clusters(X, max_clusters=10):
    """
    Find optimal number of clusters using elbow method and DB Index
    """
    db_scores = []
    inertias = []
    silhouette_scores = []
    ch_scores = []
    
    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        
        # Calculate metrics
        db_score = davies_bouldin_index(X, labels)
        inertia = kmeans.inertia_
        sil_score = silhouette_score(X, labels)
        ch_score = calinski_harabasz_score(X, labels)
        
        db_scores.append(db_score)
        inertias.append(inertia)
        silhouette_scores.append(sil_score)
        ch_scores.append(ch_score)
    
    return range(2, max_clusters + 1), db_scores, inertias, silhouette_scores, ch_scores

def plot_clustering_metrics(k_range, db_scores, inertias, silhouette_scores, ch_scores):
    """
    Plot clustering metrics for different numbers of clusters
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # DB Index
    axes[0, 0].plot(k_range, db_scores, 'bo-')
    axes[0, 0].set_xlabel('Number of Clusters')
    axes[0, 0].set_ylabel('Davies-Bouldin Index')
    axes[0, 0].set_title('Davies-Bouldin Index vs Number of Clusters')
    
    # Elbow plot
    axes[0, 1].plot(k_range, inertias, 'ro-')
    axes[0, 1].set_xlabel('Number of Clusters')
    axes[0, 1].set_ylabel('Inertia')
    axes[0, 1].set_title('Elbow Plot')
    
    # Silhouette score
    axes[1, 0].plot(k_range, silhouette_scores, 'go-')
    axes[1, 0].set_xlabel('Number of Clusters')
    axes[1, 0].set_ylabel('Silhouette Score')
    axes[1, 0].set_title('Silhouette Score vs Number of Clusters')
    
    # Calinski-Harabasz score
    axes[1, 1].plot(k_range, ch_scores, 'mo-')
    axes[1, 1].set_xlabel('Number of Clusters')
    axes[1, 1].set_ylabel('Calinski-Harabasz Score')
    axes[1, 1].set_title('Calinski-Harabasz Score vs Number of Clusters')
    
    plt.tight_layout()
    plt.savefig('clustering_metrics.png')
    plt.close()

def visualize_clusters(X, labels, feature_names, features_df):
    """
    Create enhanced visualizations of the clusters
    """
    # Create directory for plots if it doesn't exist
    import os
    if not os.path.exists('plots'):
        os.makedirs('plots')
    
    # Add cluster labels to features_df
    features_df = features_df.copy()  # Create a copy to avoid modifying the original
    features_df['Cluster'] = labels
    
    # 1. PCA Visualization with improved styling
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', alpha=0.6)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.title('Customer Segments Visualization (PCA)', pad=20)
    
    # Add cluster centers
    cluster_centers_pca = pca.transform(
        np.array([X[labels == i].mean(axis=0) for i in range(len(set(labels)))])
    )
    plt.scatter(cluster_centers_pca[:, 0], cluster_centers_pca[:, 1], 
                c='red', marker='x', s=200, linewidths=3, label='Cluster Centers')
    
    plt.colorbar(scatter, label='Cluster')
    plt.legend()
    plt.tight_layout()
    plt.savefig('plots/cluster_visualization_pca.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Enhanced Cluster Characteristics Heatmap
    cluster_means = pd.DataFrame(
        StandardScaler().fit_transform(X),
        columns=feature_names
    ).groupby(labels).mean()
    
    plt.figure(figsize=(15, 10))
    sns.heatmap(cluster_means, cmap='RdYlBu', center=0, annot=True, fmt='.2f', 
                cbar_kws={'label': 'Standardized Value'})
    plt.title('Cluster Characteristics Heatmap', pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('plots/cluster_characteristics.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # 3. Spending Distribution by Cluster
    if 'TotalSpend' in features_df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=features_df, x='Cluster', y='TotalSpend')  # Modified this line
        plt.title('Distribution of Total Spending by Cluster', pad=20)
        plt.xlabel('Cluster')
        plt.ylabel('Total Spending')
        plt.tight_layout()
        plt.savefig('plots/spending_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # 4. Transaction Count Distribution by Cluster
    if 'TransactionCount' in features_df.columns:
        plt.figure(figsize=(12, 6))
        sns.boxplot(data=features_df, x='Cluster', y='TransactionCount')  # Modified this line
        plt.title('Distribution of Transaction Count by Cluster', pad=20)
        plt.xlabel('Cluster')
        plt.ylabel('Number of Transactions')
        plt.tight_layout()
        plt.savefig('plots/transaction_distribution.png', dpi=300, bbox_inches='tight')
        plt.close()
    
    # 5. Add cluster size distribution
    plt.figure(figsize=(10, 6))
    cluster_sizes = features_df['Cluster'].value_counts().sort_index()
    cluster_sizes.plot(kind='bar')
    plt.title('Cluster Size Distribution', pad=20)
    plt.xlabel('Cluster')
    plt.ylabel('Number of Customers')
    plt.tight_layout()
    plt.savefig('plots/cluster_sizes.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return features_df  # Return the DataFrame with cluster labels
def analyze_clusters(features_df, labels, feature_names):
    """
    Perform detailed analysis of clusters
    """
    # Add cluster labels to features
    features_df['Cluster'] = labels
    
    # Initialize cluster analysis dictionary
    cluster_analysis = {}
    
    # For each cluster
    for cluster in range(len(set(labels))):
        cluster_data = features_df[features_df['Cluster'] == cluster]
        
        # Basic statistics
        stats = {
            'Size': len(cluster_data),
            'Percentage': f"{(len(cluster_data) / len(features_df) * 100):.2f}%"
        }
        
        # Feature statistics
        feature_stats = {}
        for feature in feature_names:
            if feature in cluster_data.columns:
                feature_stats[feature] = {
                    'Mean': cluster_data[feature].mean(),
                    'Median': cluster_data[feature].median(),
                    'Std': cluster_data[feature].std()
                }
        
        # Combine all statistics
        cluster_analysis[f'Cluster_{cluster}'] = {
            'Basic_Stats': stats,
            'Feature_Stats': feature_stats
        }
    
    return cluster_analysis

def generate_cluster_report(cluster_analysis, db_index, silhouette, ch_score):
    """
    Generate a detailed cluster analysis report
    """
    report = ["Customer Segmentation Analysis Report\n", "=" * 40 + "\n\n"]
    
    # Overall metrics
    report.append("Clustering Quality Metrics:\n")
    report.append(f"Davies-Bouldin Index: {db_index:.3f}")
    report.append(f"Silhouette Score: {silhouette:.3f}")
    report.append(f"Calinski-Harabasz Score: {ch_score:.3f}\n\n")
    
    # Cluster-specific analysis
    for cluster, analysis in cluster_analysis.items():
        report.append(f"{cluster} Analysis:\n{'-' * 20}\n")
        
        # Basic stats
        stats = analysis['Basic_Stats']
        report.append(f"Size: {stats['Size']} customers ({stats['Percentage']})\n")
        
        # Feature statistics
        report.append("Key Characteristics:\n")
        for feature, values in analysis['Feature_Stats'].items():
            report.append(f"{feature}:")
            report.append(f"  Mean: {values['Mean']:.2f}")
            report.append(f"  Median: {values['Median']:.2f}")
            report.append(f"  Std: {values['Std']:.2f}\n")
        
        report.append("\n")
    
    # Write report to file
    with open('cluster_analysis_report.txt', 'w') as f:
        f.write('\n'.join(report))

def main():
    # Load data
    customers_df = pd.read_csv('Customers.csv')
    transactions_df = pd.read_csv('Transactions.csv')
    
    # Prepare features
    features_df, feature_names = prepare_features(customers_df, transactions_df)
    
    # Get feature matrix (excluding CustomerID)
    X = features_df[feature_names].values
    
    # Scale features
    X = StandardScaler().fit_transform(X)
    
    # Find optimal number of clusters
    k_range, db_scores, inertias, silhouette_scores, ch_scores = find_optimal_clusters(X)
    
    # Plot metrics
    plot_clustering_metrics(k_range, db_scores, inertias, silhouette_scores, ch_scores)
    
    # Select optimal number of clusters (based on DB Index)
    optimal_k = k_range[np.argmin(db_scores)]
    
    # Perform final clustering
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    labels = kmeans.fit_predict(X)
    
    # Create visualizations and get updated features_df with cluster labels
    features_df = visualize_clusters(X, labels, feature_names, features_df)
    
    # Perform detailed cluster analysis
    cluster_analysis = analyze_clusters(features_df, labels, feature_names)
    
    # Generate detailed report
    generate_cluster_report(
        cluster_analysis,
        min(db_scores),
        silhouette_scores[optimal_k-2],
        ch_scores[optimal_k-2]
    )
    
    # Save cluster assignments
    features_df[['CustomerID', 'Cluster']].to_csv('customer_segments.csv', index=False)
    
    print("\nAnalysis complete! Check the following files:")
    print("1. plots/ directory for visualizations")
    print("2. cluster_analysis_report.txt for detailed analysis")
    print("3. customer_segments.csv for cluster assignments")
    
    return features_df, cluster_analysis

if __name__ == "__main__":
    features_df, cluster_analysis = main()


Analysis complete! Check the following files:
1. plots/ directory for visualizations
2. cluster_analysis_report.txt for detailed analysis
3. customer_segments.csv for cluster assignments
